In [156]:
import openai
import pandas as pd
import numpy as np
import yaml
import os
import openai
from openai.embeddings_utils import cosine_similarity
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import swifter
from configparser import ConfigParser
from langchain.llms import OpenAI
import langchain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI

In [155]:
api_key = "YOUR API KEY"

## Prompt Template

#### Category

In [127]:
llm = ChatOpenAI(model = 'gpt-4', temperature= .7, openai_api_key = api_key )

level1_template = """
You are an expert in understanding Retail SKUs.
What is the particular category of this SKU: {SKU} among the different kinds of categories inside snack.

Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.
"""
level1_prompt_template = PromptTemplate(input_variables=['SKU'], template = level1_template)
level1_chain = LLMChain(llm=llm, prompt= level1_prompt_template, output_key="category")

level1_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G')

'Nuts and Crackers'

#### Brand Name

In [129]:
level2_template = '''

You are an expert in understanding Retails SKUs. 
Please find out the brand name or comapny name, from the sku and category information mentioned below
sku: {SKU} 
category: {category}

Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.
'''

level2_prompt_template = PromptTemplate(input_variables=['SKU', 'category' ], template = level2_template)
level2_chain = LLMChain(llm=llm, prompt= level2_prompt_template, output_key="brand_name")

level2_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts')

'Tong Garden'

#### Pack Size

In [131]:
level3_template = '''

You are an expert in understanding Retails SKUs. 
What is the pack size for the SKU : {SKU}, given the category is {category} and brand name is {brand_name}.

Note that, different categories might have different pack sizes. Use your best judgement with the category and brand name.
Your output should be between Small, Medium, Large, Not Applicable. Return only the output and nothing else. 
Strictly review your output.
'''

level3_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name' ], template = level3_template)
level3_chain = LLMChain(llm=llm, prompt= level3_prompt_template, output_key= "pack_size")

level3_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden')

'Small'

#### Flavour

In [139]:
level4_template = '''
You are an expert in understanding Retails SKUs. 
For a given snack which belongs to category: {category} , made by this brand name: {brand_name}, available in pack size: {pack_size}.
What is the flavour of snack with SKU: {SKU}?
Flavour is something related to taste sensation.
Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.
'''

level4_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name', 'pack_size' ], template = level4_template)
level4_chain = LLMChain(llm=llm, prompt= level4_prompt_template, output_key= "flavour")

level4_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden', pack_size = 'Small' )

'Not Applicable'

#### Type

In [145]:
level5_template = '''
You are an expert in understanding Retails SKUs. 
What would be the sub-category of snack for SKU: {SKU}.
Given the category of the snack is {category} , made by brand/company {brand_name} available in pack size {pack_size}  

Here are some examples for you to understand: 

SKU: CAMEL CRACKER PEANUT 40G,
Sub-category: Peanut
SKU: COMBYWIDE KARI KACANG SEPAT 110G
Sub-category: KACANG

Return only the output and nothing else. If you don't know the answer, reply Don't Know.

Strictly review your output.
'''

level5_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name', 'pack_size' ], template = level5_template)
level5_chain = LLMChain(llm=llm, prompt= level5_prompt_template, output_key= "type")

level5_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden', pack_size = 'Small')

'Peanut'

#### Sequential Chain

In [146]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[level1_chain, level2_chain, level3_chain, level4_chain, level5_chain],
    input_variables=["SKU"],
    output_variables=['category' ,"brand_name", "pack_size", "flavour", "type"],
    verbose=True)

overall_chain({ "SKU": "PIATTOS SNACK SP 75G"})




> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'PIATTOS SNACK SP 75G',
 'category': 'Potato Chips',
 'brand_name': 'Piattos',
 'pack_size': 'Small',
 'flavour': 'Not Applicable',
 'type': 'Potato Chips'}

In [101]:
input_dataset = pd.read_excel(r"C:\Users\66660\Documents\Projects\bcn-labs-string-parsing\data\raw\input_dataset.xlsx")
sku = input_dataset['Description'].str[13:].to_list()

top25_sku = sku[0:25]

c:\Users\66660\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


#### Examples

In [147]:
overall_chain({"SKU" : top25_sku[0]})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'KOREAN COLOR CURRY POPCORN 80G',
 'category': 'Flavored Popcorn',
 'brand_name': 'Not Applicable',
 'pack_size': 'Small',
 'flavour': 'Korean Color Curry',
 'type': 'Curry Popcorn'}

In [148]:
overall_chain({"SKU" : top25_sku[1]})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G',
 'category': 'Seeds and Nuts Snacks',
 'brand_name': 'Tong Garden',
 'pack_size': 'Small',
 'flavour': 'Not Applicable',
 'type': 'Sunflower Seeds'}

In [149]:
overall_chain({"SKU" : top25_sku[2]})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'WHITE PUMPKIN SEED 500G',
 'category': 'Nuts and Seeds',
 'brand_name': 'Not Applicable',
 'pack_size': 'Medium',
 'flavour': 'Not Applicable',
 'type': 'Pumpkin Seed'}

In [150]:
a =overall_chain({"SKU" : "HWA YUAN POTATO CHIPS TAIWAN POPCORN CHICKEN FLA"})




> Entering new SequentialChain chain...

> Finished chain.


In [151]:
a

{'SKU': 'HWA YUAN POTATO CHIPS TAIWAN POPCORN CHICKEN FLA',
 'category': 'Flavored Potato Chips',
 'brand_name': 'Hwa Yuan',
 'pack_size': 'Medium',
 'flavour': 'Taiwan Popcorn Chicken',
 'type': 'Popcorn Chicken'}

In [152]:
for i in range(len(top25_sku)):
    print(overall_chain({"SKU" : top25_sku[i]}))



> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'category': 'Flavored Popcorn', 'brand_name': 'Not Applicable', 'pack_size': 'Small', 'flavour': 'Korean Color Curry', 'type': 'Curry Popcorn'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G', 'category': 'Seeds and Nuts Snacks', 'brand_name': 'Tong Garden', 'pack_size': 'Small', 'flavour': 'Not Applicable', 'type': 'Sunflower Seeds'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'WHITE PUMPKIN SEED 500G', 'category': 'Seeds & Nuts', 'brand_name': 'Not Applicable', 'pack_size': 'Medium', 'flavour': 'Not Applicable', 'type': 'Pumpkin Seed'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G', 'category': 'Vegetable Snacks', 'brand_name': 'Taisun', 'pack_size': 'Medium', 'flavour': 'Not Applicable', 'type': 'Veggie Sticks'}


> Entering new Sequenti